In [6]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly

In [74]:
def priip_rts1(df,rhp_range=np.arange(1,5),horizon=5,calcdate='20210331'):
    """
    df = DataFrame() with date and close
    rhp_range = range of rhps
    horizon = the sampling period in years
    calcdate = the calculation date
    """
    
    end_date = pd.to_datetime(calcdate)
    start_date = pd.to_datetime(calcdate)-pd.Timedelta(horizon*52,unit='W')
    idx = (df.index>=start_date)&(df.index<=end_date)
    df_rts = df.loc[idx,:].copy()
    df_ret = df_rts.pct_change().dropna(how='any')
    
    M0 = len(df_ret)
    M1 = np.mean(df_ret.values)
    M2 = np.sum((df_ret.values-M1)**2)/M0
    M3 = np.sum((df_ret.values-M1)**3)/M0
    M4 =  np.sum((df_ret.values-M1)**4)/M0
    
    mu = M1
    vol = np.sqrt(M2)
    skew = M3/(vol**3)
    kurt = M4/(vol**4)-3
    priip={}
    rhp_result = []
    mod_result  = []
    fav_result  = []
    unfav_result  = []
    stress_result  = []
    
    for rhp in rhp_range:
        N = rhp*256 #assuming 256 business days in a year
        stress_bucket =[63 if rhp>1 else 21][0]
        z = [99 if rhp<=1 else 90][0]
        za = [norm.ppf(0.01) if rhp<=1 else norm.ppf(0.05)][0]
        var =vol *np.sqrt(N)*(-1.96+0.474*skew/np.sqrt(N)-0.0687*kurt/N+0.146*(skew**2)/N)-0.5*N*(vol**2)
        VEV = (np.sqrt(3.842-2*var)-1.96)/np.sqrt(rhp)
        stressed_vol = np.percentile(df_ret.rolling(window=stress_bucket).apply(lambda x : np.std(x)).dropna(how='any').values,z)
        unfav = np.exp(M1*N+vol*np.sqrt(N)*(-1.28+0.107*skew/np.sqrt(N)+0.0724*kurt/N-0.0611*(skew**2)/N)-0.5*(vol**2)*N)
        fav = np.exp(M1*N+vol*np.sqrt(N)*(1.28+0.107*skew/np.sqrt(N)-0.0724*kurt/N+0.0611*(skew**2)/N)-0.5*(vol**2)*N)
        mod = np.exp(M1*N-vol*skew/6-0.5*(vol**2)*N)
        stress = za + ((za**2-1)/6)*skew/np.sqrt(N)+ ((za**3-3*za)/24)*kurt/N-((2*za**3-5*za)/36)*(skew**2)/N
        stress = np.exp(stressed_vol*np.sqrt(N)*stress-0.5*(stressed_vol**2)*N)
        rhp_result.append(rhp)
        fav_result.append(fav-1)
        mod_result.append(mod-1)
        unfav_result.append(unfav-1)
        stress_result.append(stress-1) 
    
    return {'rhp':rhp_result,
            'mod':mod_result,
            'fav':fav_result,
            'stress':stress_result,
            'unfav':unfav_result
           }
    
    

In [87]:
def priip_rts2(df,rhp_range=np.arange(1,5),calcdate='20210331'):
    
   
    rhp_result = []
    mod_result  = []
    fav_result  = []
    unfav_result  = []
    stress_result  = []
    
    for rhp in rhp_range:
        horizon = np.max([10,rhp+5])
       
        start_date = pd.to_datetime(calcdate)-pd.Timedelta(int(horizon*52),unit='W')
        df_rts = df.loc[(df.index>=start_date)&(df.index<=calcdate),:].copy()
        sample_weeks=(df_rts.index[-1]-df_rts.index[0]).days/7
        if sample_weeks <int(horizon*52):
            pass
        
        # create Monthly observations
        dx = pd.DataFrame({'x':1},index=pd.bdate_range(start_date,calcdate,freq='BM'),)
        df_rts = pd.concat([dx,df_rts],axis=1)
        df_rts.dropna(how='any',inplace=True)
        df_rts = pd.concat([dx,df_rts['close']],axis=1)
        df_rts['close'].fillna(method='ffill',inplace=True)
        df_rts.drop('x',axis='columns',inplace=True)
        
        #create rolling overlapping window equal to RHP returns
        periods = rhp*12
        df_ret_7a = df_rts.copy()
        df_ret_7a['end_date']=df_ret_7a.index
        df_ret_7a['start_date']=df_ret_7a['end_date'].shift(periods=periods)
        df_ret_7a['ret']= df_ret_7a['close'].rolling(window=periods,min_periods=periods).apply(lambda x : x[-1]/x[0]-1)
        df_ret_7a['start_price']= df_ret_7a['close'].rolling(window=periods,min_periods=periods).apply(lambda x :x[0])
        df_ret_7a=df_ret_7a.rename(columns={'close':'end_price'})
        df_ret_7a.dropna(how='any',inplace=True)
        
        #create rolling overlapping window from 1 year to RHP 
        min_month = 12
        max_month = rhp*12
        df_ret_7b= df_rts[-max_month-1:-min_month].copy()
        df_ret_7b['ret_raw']=0
        df_ret_7b['ret_raw']= df_rts['close'].values[-1]/df_ret_7b['close']-1
        df_ret_7b['end_price']=df_rts['close'].values[-1]
        df_ret_7b['end_date'] = df_rts.index[-1]
        df_ret_7b['start_date'] = df_ret_7b.index
        df_ret_7b['period'] = np.arange(max_month,min_month-1,-1)
        df_ret_7b['ret'] = np.power((df_ret_7b['ret_raw']+1),1/df_ret_7b['period'])
        df_ret_7b['ret'] = np.power(df_ret_7b['ret'],rhp)-1
        df_ret_7b=df_ret_7b.rename(columns={'close':'start_price'})
        
        #blending 7a and 7b together
        ret_7a_7b = np.append(df_ret_7b['ret'].values,df_ret_7a['ret'].values)
        
        mod = np.median(df_ret_7a['ret'].values)
        fav = np.max(df_ret_7a['ret'].values)
        unfav = np.min(ret_7a_7b)
        
        rhp_result.append(rhp)
        fav_result.append(fav)
        mod_result.append(mod)
        unfav_result.append(unfav)
         
        
        #calculation of the stress (same formula as the old RTS)
        end_date = pd.to_datetime(calcdate)
        start_date = pd.to_datetime(calcdate)-pd.Timedelta(horizon*52,unit='W')
        df_rts = df.loc[(df.index>=start_date)&(df.index<=end_date),:].copy()
        df_ret = df_rts.pct_change().dropna(how='any')
        M0 = len(df_ret)
        M1 = np.mean(df_ret.values)
        M2 = np.sum((df_ret.values-M1)**2)/M0
        M3 = np.sum((df_ret.values-M1)**3)/M0
        M4 =  np.sum((df_ret.values-M1)**4)/M0
    
        mu = M1
        vol = np.sqrt(M2)
        skew = M3/(vol**3)
        kurt = M4/(vol**4)-3
        
        N = rhp*256 #assuming 256 business days in a year
        stress_bucket =[63 if rhp>1 else 21][0]
        z = [99 if rhp<=1 else 90][0]
        za = [norm.ppf(0.01) if rhp<=1 else norm.ppf(0.05)][0]
        var =vol *np.sqrt(N)*(-1.96+0.474*skew/np.sqrt(N)-0.0687*kurt/N+0.146*(skew**2)/N)-0.5*N*(vol**2)
        stressed_vol = np.percentile(df_ret.rolling(window=stress_bucket).apply(lambda x : np.std(x)).dropna(how='any').values,z)
        
        stress = za + ((za**2-1)/6)*skew/np.sqrt(N)+ ((za**3-3*za)/24)*kurt/N-((2*za**3-5*za)/36)*(skew**2)/N
        stress = np.exp(stressed_vol*np.sqrt(N)*stress-0.5*(stressed_vol**2)*N)
        
        stress_result.append(stress-1)
        
        
    return {'rhp':rhp_result,
            'mod':mod_result,
            'fav':fav_result,
            'stress':stress_result,
            'unfav':unfav_result
           }
            
            

        
    

In [88]:
df = pd.read_excel('data.xlsx',sheet_name='SXXP',index_col='date')

In [89]:
calcdate = '2020-01-31'
calc1 = priip_rts1(df,rhp_range=np.arange(1,5,1),horizon=2,calcdate=calcdate)
calc2 = priip_rts2(df,rhp_range=np.arange(1,5,1),calcdate=calcdate)

In [92]:
pd.DataFrame(calc2)

,rhp,mod,fav,stress,unfav
0,1,0.074190,0.237726,-0.613183,-0.167747
1,2,0.051211,0.409461,-0.475229,-0.081818
2,3,0.135785,0.650982,-0.553660,-0.062963
3,4,0.259641,0.669418,-0.612502,-0.085515


In [91]:
calc1

{'rhp': [1, 2, 3, 4],
 'mod': [0.02884376759405405,
  0.05798522133775674,
  0.08795209129434323,
  0.11876775694002206],
 'fav': [0.19975177625371687,
  0.3153209503650374,
  0.42068992873814604,
  0.5227093691019675],
 'stress': [-0.3726127024176218,
  -0.3137648796173267,
  -0.3732725195252181,
  -0.42045723603168617],
 'unfav': [-0.11917951260841675,
  -0.15041387377350657,
  -0.16823586921004474,
  -0.17937865356980398]}

In [57]:
for x in calc1['rts1']:
    print(x['RHP']['rhp'])

4
4
4


In [53]:
calc1['rts1'][0]

{'1Y': {'stress': -2.850285401087062,
  'mod': -0.9471754609636006,
  'fav': -0.6813740188705679,
  'unfav': -1.2236036038178164,
  'rhp': 4},
 'RHP/2': {'stress': -2.1894881493695544,
  'mod': -0.8975868234988161,
  'fav': -0.518883856829758,
  'unfav': -1.286916490929057,
  'rhp': 4},
 'RHP': {'stress': -2.785197087220349,
  'mod': -0.7984095485692471,
  'fav': -0.2602160658071917,
  'unfav': -1.3472297320924846,
  'rhp': 4}}